In [104]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [105]:
########################################################

'Attrition' - Yes, No предсказываем

1) Нужно весть препоцесс в одну функцию засунуть, чтобы он работал на что угодно.

Loading dataset.

In [106]:
df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')
df_train[0:15]

,EmployeeID,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,...,RelationshipSatisfaction,StandardHours,Shift,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,1317087,40,No,Travel_Rarely,1398,Cardiology,2,4,Life Sciences,1,...,4,80,0,21,2,3,20,15,1,12
1,1548175,40,No,Travel_Rarely,1300,Maternity,24,2,Technical Degree,1,...,1,80,2,9,3,3,9,8,4,7
2,1215433,25,No,Travel_Rarely,622,Cardiology,13,1,Medical,1,...,3,80,0,7,1,3,7,4,0,6
3,1375351,33,No,Travel_Rarely,922,Maternity,1,5,Medical,1,...,3,80,1,10,2,3,6,1,0,5
4,1028734,39,No,Travel_Frequently,505,Maternity,2,4,Technical Degree,1,...,4,80,0,20,1,3,19,6,11,8
5,1863408,27,No,Non-Travel,210,Cardiology,1,1,Marketing,1,...,4,80,1,6,0,3,5,4,1,4
6,1316836,32,No,Non-Travel,1200,Maternity,1,4,Technical Degree,1,...,2,80,1,9,3,2,6,5,1,2
7,1574111,33,No,Travel_Rarely,392,Cardiology,2,4,Medical,1,...,3,80,2,6,5,3,6,2,0,4
8,1143717,34,No,Travel_Rarely,511,Cardiology,3,2,Life Sciences,1,...,1,80,0,6,3,3,2,2,2,2
9,1474227,25,No,Travel_Rarely,383,Cardiology,9,2,Life Sciences,1,...,1,80,0,6,2,3,3,2,2,2


This function transforms all the data into numerical values

In [107]:
def data_renaming(df, renames_dict, features=None):
    if features is None:
        df = df.replace({**renames_dict})
    else:
        for i, feature in enumerate(features):
            df[feature] = df[feature].replace({**renames_dict[i]})
    return   df

Preprocessing data: сщтмукештп all string values into numerical values.

In [108]:
features = [
    'BusinessTravel',
    'Department',
    'EducationField',
    'Gender',
    'JobRole',
    'MaritalStatus',
    'OverTime',
]
renames = [
    {'Travel_Rarely': 0.5, 'Travel_Frequently': 1, 'Non-Travel': 0},
    {'Cardiology': 0, 'Maternity': 1, 'Neurology': 2},
    {'Life Sciences': 0, 'Medical': 1, 'Marketing': 2, 'Technical Degree': 3,
     'Human Resources': 4, 'Other': 5},
    {'Male': 0, 'Female': 1},
    {'Nurse': 1, 'Other': 0, 'Therapist': 2, 'Administrative': 3, 'Admin': 4},
    {'Married': 0, 'Single': 1, 'Divorced': 2},
    {'No': 0, 'Yes' : 1},
]
data_train_num = data_renaming(df_train, renames, features)
data_train_num = data_renaming(df_train, [{'No': 0, 'Yes' : 1}], ['Attrition'])
data_test_num = data_renaming(df_test, renames, features)

Deleting ['Over18', 'EmployeeCount', 'StandardHours']  since they carry 0 info (all the same).

In [109]:
def delete_useless_data(data, features):
    return data.drop(features, axis=1)

features = ['Over18', 'EmployeeCount', 'StandardHours']
data_train_num_useful = delete_useless_data(data_train_num, features)
data_test_num_useful = delete_useless_data(data_test_num, features)

Rescaling all the features into [0, 1] diapason.

In [113]:
from sklearn.preprocessing import MinMaxScaler

def rescaler_data(data, data_train, range=(0,1)):
    min_max_scaler = MinMaxScaler(feature_range=range)
    min_max_scaler.fit(data_train)
    return min_max_scaler.transform(data)

X_train_num_useful = data_train_num_useful.drop('Attrition', axis=1)
y_train = data_train_num_useful['Attrition']
X_train = rescaler_data(X_train_num_useful, X_train_num_useful)
X_test = rescaler_data(data_test_num_useful, X_train_num_useful)
print(X_test)

[[0.70043927 0.57142857 0.5        ... 0.16666667 0.         0.23529412]
 [0.73527667 0.26190476 0.5        ... 0.         0.         0.        ]
 [0.57519093 0.47619048 0.         ... 0.38888889 0.46666667 0.29411765]
 ...
 [0.712923   0.26190476 0.5        ... 0.11111111 0.13333333 0.11764706]
 [0.61017463 0.38095238 0.         ... 0.11111111 0.06666667 0.11764706]
 [0.12573255 0.45238095 0.5        ... 0.05555556 0.         0.11764706]]


Looking at the correlations between different features.